In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

import tensorflow as tf
import keras

Using TensorFlow backend.


In [0]:
from keras.applications import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, load_img

from sklearn import metrics
from sklearn.model_selection import KFold

In [0]:
train_dir = '/content/gdrive/My Drive/tinder-bot/Images'

image_size = 200


In [0]:
train_indexer = pd.read_csv('/content/gdrive/My Drive/tinder-bot/All_Ratings.csv')
train_indexer.head()

,rater,filename,rating
0,1,ftw1.jpg,3
1,2,ftw1.jpg,4
2,3,ftw1.jpg,4
3,4,ftw1.jpg,5
4,5,ftw1.jpg,4


In [0]:
current = train_indexer.iloc[0, 1]
previous = current
print(current)

ftw1.jpg


In [0]:
ratings = pd.DataFrame(columns=['filename', 'rating'])
rating_sum = 0

for i, row in train_indexer.iterrows():
  current = row[1]
  if current == previous:
    rating_sum += float(row[2])
  else:
    rating = str(rating_sum / 60)
    ratings = ratings.append({'filename': previous, 'rating': rating}, ignore_index=True)
    rating_sum = 0
  previous = current

In [0]:
ratings.head(10)

,filename,rating
0,ftw1.jpg,4.116666666666666
1,ftw10.jpg,3.5
2,ftw100.jpg,2.1166666666666667
3,ftw101.jpg,2.45
4,ftw102.jpg,3.25
5,ftw103.jpg,2.566666666666667
6,ftw104.jpg,2.5
7,ftw105.jpg,2.716666666666667
8,ftw106.jpg,2.85
9,ftw107.jpg,3.7666666666666666


In [0]:
ratings.to_csv('/content/gdrive/My Drive/tinder-bot/avg_ratings.csv')

In [0]:
def extract_face(image, face_x,	face_y,	face_width,	face_height):
  return image[face_y:face_y+face_height, face_x:face_x+face_width]

In [0]:
def train_vgg_model(training_set, validation_set):

  train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

  train_generator = train_datagen.flow_from_dataframe(
        dataframe=training_set,
        directory=train_dir,
        x_col="filename",
        y_col="rating",
        target_size=(image_size, image_size),
        batch_size=100,
        class_mode="sparse")

  validation_datagen = ImageDataGenerator(rescale=1./255)

  validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=validation_set,
        directory=train_dir,
        x_col="filename",
        y_col="rating",
        target_size=(image_size, image_size),
        batch_size=30,
        class_mode="sparse")

  #Load the VGG model
  vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size, image_size, 3))

  # Freeze all layers except the last 6 layers
  for layer in vgg_conv.layers[:-6]:
    layer.trainable = False

  # Create the model
  vgg_model = models.Sequential()

  # Add the vgg convolutional base model
  vgg_model.add(vgg_conv)

  # Add new layers
  vgg_model.add(layers.Flatten())
  vgg_model.add(layers.Dense(1024, activation='relu'))
  vgg_model.add(layers.Dropout(0.5))
  vgg_model.add(layers.Dense(1))

  # Show a summary of the model
  vgg_model.summary()

  # Compile the model
  vgg_model.compile(loss='mae', optimizer='adam')
  
  # Train the Model
  vgg_history = vgg_model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
  
  return vgg_model, vgg_history

In [0]:
vgg_model, vgg_history = train_vgg_model(test_data_train, test_data_valid)

In [0]:
# Create K-fold divider
kf = KFold(3, shuffle=True, random_state=42)


In [0]:
# Divide and train each fold
fold = 0
model = []
history = []
for train, test in kf.split(ratings, ratings['rating']):
  fold += 1
  print(f"Fold: #{fold}")

  train_set = ratings.iloc[train]
  test_set = ratings.iloc[test]

  vgg_model, vgg_history = train_vgg_model(train_set, test_set)

  models.append(vgg_model)
  history.append(vgg_history)


Fold: #1
Found 3666 validated image filenames belonging to 197 classes.
Found 1833 validated image filenames belonging to 191 classes.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 1025      
Total params: 33,591,105
Trainable params: 28,315,649
Non-trainable params: 5,275,456
_______________________________

In [0]:
# Use sklearn metrics to evaluate all folds